In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os

df_station = pd.read_csv('./TempHisStation_NEW.csv')

In [ ]:
# df_station.info

In [ ]:
folder = "Temp3Hr"
path = f"./{folder}/" #資料夾目錄
files= os.listdir(path) #得到資料夾下的所有檔名稱
df = pd.read_csv(f'./{folder}/{files[0]}',index_col=[x for x in range(24)]) #1天24小時

In [ ]:
# 重制index，變成我能看懂的
df = df.reset_index()

In [ ]:
#修改columns名稱，用第1列來表示
df.columns = list(df.iloc[0])

In [ ]:
# 刪除前兩列說明
df = df.drop(index=[0,1])

In [ ]:
# 加入一行，若是加入一列要用df.loc[index名稱]=
df['觀測站'] = files[0][:6]
df['觀測時間'] = files[0][7:17]
df['觀測時間'] = df['觀測時間']+'-'+df['觀測時間(hour)']

In [ ]:
# 用for把剩餘的csv全加進去
files = os.listdir(f'./{folder}/')

for file in files[1:]:
    if not os.path.isdir(file): #判斷是否是資料夾，不是資料夾才開啟
        df_tmp = pd.read_csv(f'./{folder}/{files[files.index(file)]}',index_col=[x for x in range(24)])
        df_tmp = df_tmp.reset_index()
        df_tmp.columns = list(df_tmp.iloc[0])
        df_tmp = df_tmp.drop(index=[0,1])
        df_tmp['觀測站'] = file[:6]
        df_tmp['觀測時間'] = file[7:17]
        df_tmp['觀測時間'] = df_tmp['觀測時間']+'-'+df_tmp['觀測時間(hour)']
        df = pd.concat([df,df_tmp])

In [ ]:
df.index = df['觀測站']

df = df.drop(columns=['觀測時間(hour)'])

In [ ]:
#刪除行
df = df.drop(['測站氣壓(hPa)'],axis=1)
df = df.drop(['海平面氣壓(hPa)'],axis=1)
df = df.drop(['露點溫度(℃)'],axis=1)
df = df.drop(['風向(360degree)'],axis=1)
df = df.drop(['最大陣風(m/s)'],axis=1)
df = df.drop(['最大陣風風向(360degree)'],axis=1)
df = df.drop(['降水時數(h)'],axis=1)
df = df.drop(['日照時數(h)'],axis=1)
df = df.drop(['全天空日射量(MJ/㎡)'],axis=1)
df = df.drop(['能見度(km)\t'],axis=1)
df = df.drop(['紫外線指數'],axis=1)
df = df.drop(['總雲量(0~10)'],axis=1)

In [ ]:
df.head(1)

In [ ]:
#改index名稱
df_station.index = df_station['站號']

In [ ]:
df_station.head(1)

In [ ]:
df1 = df.iloc[:50000,:]
df2 = df.iloc[50000:100000,:]
df3 = df.iloc[100000:150000,:]
df4 = df.iloc[150000:200000,:]
df5 = df.iloc[200000:250000,:]
df6 = df.iloc[250000:300000,:]
df7 = df.iloc[300000:350000,:]
df8 = df.iloc[350000:400000,:]
df9 = df.iloc[400000:,:]

In [ ]:
#新增地點說明欄位資訊
df1['站名'] = [df_station.loc[df1.iloc[i]['觀測站']]['站名'] for i in range(len(df1))]
df1['經度'] = [df_station.loc[df1.iloc[i]['觀測站']]['經度'] for i in range(len(df1))]
df1['緯度'] = [df_station.loc[df1.iloc[i]['觀測站']]['緯度'] for i in range(len(df1))]
df1['城市'] = [df_station.loc[df1.iloc[i]['觀測站']]['城市'] for i in range(len(df1))]
df1['地址'] = [df_station.loc[df1.iloc[i]['觀測站']]['地址'] for i in range(len(df1))]

In [ ]:
# 把nan換成空值，nan是float，不能用==判斷
df1.fillna('', inplace=True)

In [ ]:
lst_sql = df1.values.tolist()
        
# 轉成list的資料不知為何出現\xa0結尾，故把它幹掉
for i in range(len(lst_sql)):
    for j in range(5):
        lst_sql[i][j] = lst_sql[i][j].replace(u'\xa0', u'')

#...資料變空值
for i in range(len(lst_sql)):
    for j in range(5):
        lst_sql[i][j] = lst_sql[i][j].replace(u'...', u'')

for i in range(len(lst_sql)):
    for j in range(5):
        lst_sql[i][j] = lst_sql[i][j].replace(u'X', u'')

for i in range(len(lst_sql)):
    for j in range(5):
        lst_sql[i][j] = lst_sql[i][j].replace(u'T', u'')
        
for i in range(len(lst_sql)):
    for j in range(5):
        lst_sql[i][j] = lst_sql[i][j].replace(u'/', u'')
        
for i in range(len(lst_sql)):
    for j in range(5):
        lst_sql[i][j] = lst_sql[i][j].replace(u'&', u'')

In [ ]:
import pyodbc

#連線
server = 'sqyhi03az.database.windows.net'
username = 'irentadmin'
password = 'hims@QaWsEd'
database = 'IRENT_V2'
driver = '{ODBC Driver 17 for SQL Server}'
# connectionString = 'DRIVER={0};PORT=1433;SERVER={1};DATABASE={2};UID={3};PWD={4}'.format(driver, server, database, username, password)
connectionString = f'DRIVER={driver};PORT=1433;SERVER={server};DATABASE={database};UID={username};PWD={password}'
cnxn = pyodbc.connect(connectionString)
cursor = cnxn.cursor()

for i in range(len(lst_sql)):
    with open("log.txt","a") as file: #寫入不覆蓋，一直寫下去
        file.write(f'{lst_sql[i]}\n')
        
    try:
        if lst_sql[i][5]!='':
            insertSql = f"insert into TB_WeatherHis(day,temp,wet,windspeed,rain,station,stationNAME,lon,lat,city,address,SYSDT,PROCD)\
            values ('{lst_sql[i][5]}','{lst_sql[i][0]}','{lst_sql[i][1]}','{lst_sql[i][2]}','{lst_sql[i][3]}','{lst_sql[i][4]}'\
            ,'{lst_sql[i][6]}','{lst_sql[i][7]}','{lst_sql[i][8]}','{lst_sql[i][9]}','{lst_sql[i][10]}',getdate(),1)"
            cursor.execute(insertSql) # 執行
    except:
        continue
    
cursor.commit()
cnxn.close()